In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

df1 = pd.read_csv("./마포구_test.csv")
street_name = df1['마포구 도로명']
street_val = street_name.values


# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)


# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait


# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

for i, keyword in enumerate(df1['마포구 도로명'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df1.shape[0] -1} 행", keyword)
    naver_map_search_url = f"https://map.naver.com/v5/search/서울시 {keyword} 카페"
    
    driver.get(naver_map_search_url)
    time.sleep(3.5)

    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다

    sleep(1)

    if driver.find_element_by_class_name("summary_title"):
        print("find")

    # frame 변경 메소드
    def switch_frame(frame):
        driver.switch_to.default_content()  # frame 초기화
        driver.switch_to.frame(frame)  # frame 변경
        res
        soup


    # 페이지 다운
    def page_down(num):
        body = driver.find_element_by_css_selector('body')
        body.click()
        for i in range(num):
            body.send_keys(Keys.PAGE_DOWN)


    # frame 변경
    switch_frame('searchIframe')
    page_down(40)
    sleep(5)

    # 매장 리스트
    store_list = driver.find_elements_by_css_selector('._22p-O') #driver.find_elements_by_css_selector('._1EKsQ')
    # 페이지 리스트
    next_btn = driver.find_elements_by_css_selector('._2ky45 > a') #동일

    # dictionary 생성
    store_dict = {'매장정보': []}
    # 시작시간
    start = time.time()
    print('[크롤링 시작...]')

    # 크롤링 (페이지 리스트 만큼)
    for btn in range(len(next_btn))[1:-1]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
        print(len(next_btn))###test
        for data in range(len(store_list)):  # 매장 리스트 만큼
            page = driver.find_elements_by_css_selector('._3Apve') #driver.find_elements_by_css_selector('.OXiLu')
            print(data)###test
            print(len(store_list))###test
            page[data].click()
            sleep(2)
            try:
                # 상세 페이지로 이동
                switch_frame('entryIframe')
                time_wait(5, '._3XamX')
                # 스크롤을 맨밑으로 1초간격으로 내린다.
                for down in range(3):
                    sleep(1)
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # -----매장명 가져오기-----
                store_name = driver.find_element_by_css_selector('._3XamX').text
                print(store_name)

                # -----카테고리 가져오기-----
                store_category = driver.find_element_by_css_selector('._3ocDE').text
                print(store_category)

                # -----주소(위치)-----
                try:
                    store_addr_list = driver.find_elements_by_css_selector('._1aj6-')
                    for i in store_addr_list:
                        store_addr = i.find_element_by_css_selector('._1h3B_').text
                except:
                    pass
                print(store_addr)
                # -----전화번호 가져오기-----
                try:
                    store_tel = driver.find_element_by_css_selector('._3ZA0S').text
                except:
                    pass

                print(store_tel)
                # -----영업시간-----
                try:
                    store_time_list = driver.find_elements_by_css_selector('._2vK84')  # 아니 태그가 그세 바뀌네ㅡ,.ㅡ
                    for i in store_time_list:
                        store_time = i.find_element_by_css_selector('._3uEtO > time').text
                except:
                    pass
                print(store_time)

                # -----썸네일 사진 주소-----
                try:
                    thumb_list = driver.find_element_by_css_selector('.cb7hz') \
                        .value_of_css_property('background-image')  # css 속성명을 찾는다
                    store_thumb = re.sub('url|"|\)|\(', '', thumb_list)  # url , (" ") 제거
                except:
                    pass
                print(store_thumb)

                # ---- dict에 데이터 집어넣기----
                dict_temp = {
                    'name': store_name,
                    'category' : store_category,
                    'tel': store_tel,
                    'addr': store_addr,
                    'time': store_time,
                    'thumb': store_thumb
                }

                store_dict['매장정보'].append(dict_temp)
                store_name = None
                store_category = None
                store_tel = None
                store_addr = None
                store_time = None
                store_thumb = None

                print(f'{store_name} ...완료')
                switch_frame('searchIframe')
                sleep(1)

            except:
                print('ERROR!' * 3)

        # 다음 페이지 버튼
        if page[-1]:  # 마지막 매장일 경우 다음버튼 클릭
            next_btn[-1].click()
            sleep(2)
            store_list = driver.find_elements_by_css_selector('._22p-O')# 매장 리스트
        else:
            print('페이지 인식 못함')
            break

    print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을닫는다.

# json 파일로 저장
with open('data/store_data.json', 'w', encoding='utf-8') as f:
    json.dump(store_dict, f, indent=4, ensure_ascii=False)

C:\Users\admin\AppData\Local\Temp\ipykernel_21816\64718544.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로


이번에 찾을 키워드 : 0 / 232 행 가양대로


C:\Users\admin\AppData\Local\Temp\ipykernel_21816\64718544.py:54: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  if driver.find_element_by_css_selector("icon_today"):


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"icon_today"}
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CD7413+2389011]
	Ordinal0 [0x00C69F61+1941345]
	Ordinal0 [0x00B5C658+837208]
	Ordinal0 [0x00B891DD+1020381]
	Ordinal0 [0x00B8949B+1021083]
	Ordinal0 [0x00BB6032+1204274]
	Ordinal0 [0x00BA4194+1130900]
	Ordinal0 [0x00BB4302+1196802]
	Ordinal0 [0x00BA3F66+1130342]
	Ordinal0 [0x00B7E546+976198]
	Ordinal0 [0x00B7F456+980054]
	GetHandleVerifier [0x00E89632+1727522]
	GetHandleVerifier [0x00F3BA4D+2457661]
	GetHandleVerifier [0x00D6EB81+569713]
	GetHandleVerifier [0x00D6DD76+566118]
	Ordinal0 [0x00C70B2B+1968939]
	Ordinal0 [0x00C75988+1989000]
	Ordinal0 [0x00C75A75+1989237]
	Ordinal0 [0x00C7ECB1+2026673]
	BaseThreadInitThunk [0x758FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77A77A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77A77A4E+238]


csv 파일 불러오기

In [66]:
import pandas as pd

df1 = pd.read_csv("./도로명_조회_목록.csv")
street_name = df1['마포구 도로명']
street_val = street_name.values
for i in street_val:
    print(i + " 카페")

가양대로 카페
강변북로 카페
고산길 카페
고산2길 카페
고산3길 카페
고산5길 카페
고산7길 카페
고산11길 카페
고산12길 카페
고산14길 카페
고산16길 카페
고산17길 카페
고산18길 카페
광성로 카페
광성로1길 카페
광성로4길 카페
광성로6길 카페
광성로6안길 카페
구룡길 카페
국회대로 카페
굴레방로 카페
굴레방로1길 카페
굴레방로3길 카페
굴레방로5길 카페
굴레방로7길 카페
굴레방로9길 카페
농수산시장로 카페
대흥로 카페
대흥로18가길 카페
대흥로24가길 카페
대흥로2길 카페
대흥로4길 카페
대흥로6길 카페
대흥로8길 카페
대흥로9길 카페
대흥로11길 카페
대흥로13길 카페
대흥로14길 카페
대흥로18길 카페
대흥로20길 카페
대흥로21길 카페
대흥로22길 카페
대흥로24길 카페
대흥로25길 카페
대흥로27길 카페
대흥로28길 카페
대흥로29길 카페
대흥로30길 카페
대흥로18나길 카페
대흥로24나길 카페
대흥로24다길 카페
대흥로24라길 카페
대흥로24마길 카페
대흥로24바길 카페
대흥로9안길 카페
대흥로11안길 카페
대흥로20안길 카페
대흥로29안길 카페
도화길 카페
도화2길 카페
도화4길 카페
도화2안길 카페
독막로 카페
독막로2길 카페
독막로3길 카페
독막로4길 카페
독막로5길 카페
독막로6길 카페
독막로7길 카페
독막로8길 카페
독막로9길 카페
독막로12길 카페
독막로14길 카페
독막로15길 카페
독막로18길 카페
독막로19길 카페
독막로20길 카페
독막로21길 카페
독막로22길 카페
독막로24길 카페
독막로25길 카페
독막로26길 카페
독막로27길 카페
독막로28길 카페
독막로29길 카페
독막로31길 카페
독막로32길 카페
독막로34길 카페
독막로36길 카페
독막로38길 카페
독막로40길 카페
독막로42길 카페
독막로20나길 카페
독막로12안길 카페
독막로32안길 카페
동교로 카페
동교로1길 카페
동교로3길 카페
동교로5길 카페
동교로8길 카페
동교로9길 카페
동교로12길 카페
동교로13길 카페
동교로15길 카페
동교로16길 카페
동교로17

영업시간 크롤링 테스트

In [29]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '고양시 강선로 카페'  # 검색어


# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait


# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# 검색창 찾기
search = driver.find_element_by_css_selector('div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다

sleep(1)


# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    res
    soup


# 페이지 다운
def page_down(num):
    body = driver.find_element_by_css_selector('body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)


# frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(5)

# 매장 리스트
store_list = driver.find_elements_by_css_selector('._22p-O') #driver.find_elements_by_css_selector('._1EKsQ')
# 페이지 리스트
next_btn = driver.find_elements_by_css_selector('._2ky45 > a') #동일

# dictionary 생성
store_dict = {'매장정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    store_list
    for data in range(len(store_list)):  # 매장 리스트 만큼
        page = driver.find_elements_by_css_selector('._3Apve') #driver.find_elements_by_css_selector('.OXiLu')
        page[data].click()
        sleep(2)
        try:
            # 상세 페이지로 이동
            switch_frame('entryIframe')
            time_wait(5, '._3XamX')
            # 스크롤을 맨밑으로 1초간격으로 내린다.
            for down in range(3):
                sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # -----매장명 가져오기-----
            store_name = driver.find_element_by_css_selector('._3XamX').text
            print(store_name)

            # -----카테고리 가져오기-----
            store_category = driver.find_element_by_css_selector('._3ocDE').text
            print(store_category)

            # -----주소(위치)-----
            try:
                store_addr_list = driver.find_elements_by_css_selector('._1aj6-')
                for i in store_addr_list:
                    store_addr = i.find_element_by_css_selector('._1h3B_').text
            except:
                pass
            print(store_addr)
            # -----전화번호 가져오기-----
            try:
                store_tel = driver.find_element_by_css_selector('._3ZA0S').text
            except:
                pass

            print(store_tel)
            # -----영업시간-----
            try:
                store_time_list = driver.find_elements_by_css_selector('._2vK84')  # 아니 태그가 그세 바뀌네ㅡ,.ㅡ
                for i in store_time_list:
                    store_time = i.find_element_by_css_selector('._3uEtO > time').text
            except:
                pass
            print(store_time)

            html = driver.page_source 
            soup = BeautifulSoup(html, 'html.parser') 
            # print(soup) 
            notices = soup.select('바꿔') 

            resultlist = [] 

            for n in notices: 
                # print ('aria-hidden-', n['aria-hidden']) aria-expanded로 바꿔
                if n['aria-expanded']=='false': 
                    lis = n.select('li') 
                    for l in lis: 
                        result = dict() 
                        result['rank'] = l.select_one('.num_issue').text 
                        result['title']= l.select_one('.txt_issue').text 
                        result['url'] = l.a['href'] 
                        # print(l.select_one('.num_issue').text) 
                        # print(l.select_one('.txt_issue').text) 
                        # print('href=',l.a['href']) 
                        resultlist.append(result) 

            # -----썸네일 사진 주소-----
            try:
                thumb_list = driver.find_element_by_css_selector('.cb7hz') \
                    .value_of_css_property('background-image')  # css 속성명을 찾는다
                store_thumb = re.sub('url|"|\)|\(', '', thumb_list)  # url , (" ") 제거
            except:
                pass
            print(store_thumb)

            # ---- dict에 데이터 집어넣기----
            dict_temp = {
                'name': store_name,
                'category' : store_category,
                'tel': store_tel,
                'addr': store_addr,
                'time': store_time,
                'thumb': store_thumb
            }

            store_dict['매장정보'].append(dict_temp)
            store_name = None
            store_category = None
            store_tel = None
            store_addr = None
            store_time = None
            store_thumb = None

            print(f'{store_name} ...완료')
            switch_frame('searchIframe')
            sleep(1)

        except:
            print('ERROR!' * 3)

    # 다음 페이지 버튼
    if page[-1]:  # 마지막 매장일 경우 다음버튼 클릭
        next_btn[-1].click()
        sleep(2)
    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을닫는다.

# json 파일로 저장
with open('data/store_data.json', 'w', encoding='utf-8') as f:
    json.dump(store_dict, f, indent=4, ensure_ascii=False)

C:\Users\admin\AppData\Local\Temp\ipykernel_25408\591872428.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\591872428.py:38: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search = driver.find_element_by_css_selector('div.input_box > input.input_search')
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\591872428.py:58: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body = driver.find_element_by_css_selector('body')
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\591872428.py:70: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_list = driver.find_elements_by

[크롤링 시작...]


C:\Users\admin\AppData\Local\Temp\ipykernel_25408\591872428.py:97: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  store_name = driver.find_element_by_css_selector('._3XamX').text
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\591872428.py:101: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  store_category = driver.find_element_by_css_selector('._3ocDE').text
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\591872428.py:106: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_addr_list = driver.find_elements_by_css_selector('._1aj6-')
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\591872428.py:114: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value

요거트월드 주엽점
카페,디저트
경기 고양시 일산서구 강선로 118 상가 102호 요거트월드
3주엽역 8번 출구에서903m
0507-1309-3968
새벽 02:00에 영업 종료
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220305_176%2F164647169135604wYF_JPEG%2FC9DED292-5D13-49ED-9EB3-929D007D5528.jpeg
None ...완료
커피팩토리 후곡
카페,디저트
경기 고양시 일산서구 강선로 140 후곡마을17단지 태영상가 1층
경의중앙일산역 1번 출구에서950m
031-911-0564
20:30에 라스트오더
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210122_145%2F1611275077148G6Q2l_JPEG%2FKakaoTalk_20210122_092330200.jpg
None ...완료
감성커피 일산주엽점
카페,디저트
경기 고양시 일산서구 강선로 59
3주엽역 8번 출구에서258m
031-913-3888
20:30에 라스트오더
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20211031_171%2F16356746645344p4tJ_JPEG%2FY36Cpd3BY7xq_m4HSKfo5eTf.jpeg.jpg
None ...완료
루루라라 베이커리
베이커리
경기 고양시 일산서구 강선로 32 강선15단지 상가동 105호
3주엽역 1번 출구에서413m
0507-1367-8621
11:00에 영업 시작
https://search.pstatic.net/co